In [1]:
# Ignore SQLITE warnings related to Decimal numbers in the housing database.
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import Dependencies.
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [3]:
# Create an engine for the housing.sqlite database.
engine = create_engine("sqlite:///../Resources/housingUpdated.sqlite")

In [4]:
# Reflect Database into ORM classes.
Base = automap_base()
Base.prepare(engine, reflect=True)

In [5]:
# Create a database session object.
session = Session(engine)

In [6]:
# Get the table names of the database.
inspector = inspect(engine)
inspector.get_table_names()

['listings']

In [7]:
# Get the names and types of the columns for the table listings.
columns = inspector.get_columns("listings")
for c in columns:
    print(c["name"], c["type"])


id INTEGER
address VARCHAR(255)
price INTEGER
home_type VARCHAR(255)
bedrooms INTEGER
bathrooms FLOAT
square_feet INTEGER
built INTEGER
lot_size FLOAT
neighborhood VARCHAR(255)
county VARCHAR(255)
city VARCHAR(255)
zipcode INTEGER
high_school VARCHAR(255)
middle_school VARCHAR(255)
elementary_school VARCHAR(255)


In [8]:
# Save a reference to the listings table as "Listings".
Listings = Base.classes.listings

In [9]:
# Count the number of entries in the listings table.
listings_count = session.query(Listings).count()
listings_count

1830

In [10]:
# Count the number of distinct home types found in the listings table.
home_types_count = session.query(Listings.home_type).distinct().count()
home_types_count
# Given the sheer number of categories, this seems a good variable to drop...

72

In [11]:
# City, county, and neighborhood would seem to be specified by zip code. Count the number of distinct zip codes.
zipcode_count = session.query(Listings.zipcode).distinct().count()
zipcode_count

33

In [12]:
# Count the number of schools for each level.
hs_count = session.query(Listings.high_school).distinct().count()
ms_count = session.query(Listings.middle_school).distinct().count()
es_count = session.query(Listings.elementary_school).distinct().count()
hs_count, ms_count, es_count
# There are an awful lot of these, these categories may need to be removed when training models...

(25, 62, 115)

In [13]:
# Examine the values in high_school for potentially erroneous data.
high_school_values = session.query(Listings.high_school).distinct().all()
high_school_values

[('Current Price:'),
 ('Reynolds'),
 ('Centennial'),
 ('David Douglas'),
 ('Other'),
 ('Lincoln'),
 ('Jefferson'),
 ('Sunset'),
 ('Beaverton'),
 ('Roosevelt'),
 ('Wilson'),
 ('Madison'),
 ('Westview'),
 ('Franklin'),
 ('Cleveland'),
 ('Tualatin'),
 ('Grant'),
 ('Parkrose'),
 ('Tigard'),
 ('Milwaukie'),
 ('Southridge'),
 ('Liberty'),
 ('Riverdale'),
 ('Scappoose'),
 ('Lake Oswego')]

In [18]:
# Remove rows where high school is listed as "Current Price:".
deletion = Listings.__table__.delete().where(Listings.high_school=="Current Price:")
engine.execute(deletion)

In [19]:
# Check the high school values again to confirm deletion.
high_school_values = session.query(Listings.high_school).distinct().all()
high_school_values

[('Reynolds'),
 ('Centennial'),
 ('David Douglas'),
 ('Other'),
 ('Lincoln'),
 ('Jefferson'),
 ('Sunset'),
 ('Beaverton'),
 ('Roosevelt'),
 ('Wilson'),
 ('Madison'),
 ('Westview'),
 ('Franklin'),
 ('Cleveland'),
 ('Tualatin'),
 ('Grant'),
 ('Parkrose'),
 ('Tigard'),
 ('Milwaukie'),
 ('Southridge'),
 ('Liberty'),
 ('Riverdale'),
 ('Scappoose'),
 ('Lake Oswego')]

In [13]:
# Close the session.
session.close()